In [98]:
import requests
import pandas as pd
import discord
from discord.ext import commands

base_url = 'https://api.hypixel.net/'
add_url = 'v2/skyblock/auctions'
url = base_url + add_url
# Function to fetch data from Polygon.io
response = requests.get(url)
auction_data = response.json()
auction_data

{'success': True,
 'page': 0,
 'totalPages': 62,
 'totalAuctions': 61439,
 'lastUpdated': 1720914332239,
 'auctions': [{'uuid': 'f609995e24e3432e84079743da745053',
   'auctioneer': '5dbbace6cf2f4a45a33c10c115afec63',
   'profile_id': 'a010e9ca156449268b60e127c813a80d',
   'coop': ['5dbbace6cf2f4a45a33c10c115afec63'],
   'start': 1720914331765,
   'end': 1721000731765,
   'item_name': 'Magic Find Enrichment',
   'item_lore': '§7§8Combinable in Anvil\n\n§7Enriches an accessory with the\n§7power of §b+0.5✯ Magic Find §7when\n§7applied to a §6Legendary §7or §dMythic\n§d§7accessory.\n\n§7Only §7§lONE §7enrichment can be applied\n§7to a given accessory.\n\n§c§lSPECIAL',
   'extra': 'Magic Find Enrichment Skull Item',
   'category': 'misc',
   'tier': 'SPECIAL',
   'starting_bid': 7400000,
   'item_bytes': 'H4sIAAAAAAAAAFVTy46jRhQt9/QktqMoWSabiEiziRK3wBi7WWSBwW7jccG0jR+waRVFNZSbh8XD2Ci/ECkfMHvvkn/wp+RDohRkEnVYwOWee885t+B2AeiAFu0CAFo34IZ6rV9b4K2aFHHe6oI3OfI74JbEOAD19QZ0ZtQj0xD5GXv9qwu6q5ciDM0y

In [99]:
# Data cleaning
auctions_data = auction_data['auctions']
df = pd.DataFrame(auctions_data)
df.drop(columns=['uuid', 'auctioneer','profile_id','coop', 'extra','category','item_lore','tier','item_bytes','claimed','claimed_bidders','item_uuid','bin','bids'],inplace=True)
df


,start,end,item_name,starting_bid,highest_bid_amount,last_updated
0,1720914331765,1721000731765,Magic Find Enrichment,7400000,0,1720914331765
1,1720914331760,1720935931760,Pickonimbus 2000,199000,0,1720914331760
2,1720914331645,1721087131645,Terror Helmet,170000,0,1720914331645
3,1720914331568,1720935931568,Cicada Symphony Vinyl,650000,0,1720914331568
4,1720914331531,1720935931531,Gemstone Chamber,6700000,0,1720914331531
...,...,...,...,...,...,...
995,1720830085067,1720916485067,Auspicious Gemstone Gauntlet,500,17000000,1720914047306
996,1720914046602,1720935646602,Blue Gift Backpack Skin,2500000,0,1720914046602
997,1720914046328,1720935646328,Sweet Axe,100000,0,1720914046328
998,1720914046200,1721259646200,Zombie Chestplate,889999,0,1720914046200


In [100]:
df['start'] = pd.to_datetime(df['start'], unit='ms')
df['end'] = pd.to_datetime(df['end'], unit='ms')
df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
df

,start,end,item_name,starting_bid,highest_bid_amount,last_updated
0,2024-07-13 23:45:31.765,2024-07-14 23:45:31.765,Magic Find Enrichment,7400000,0,2024-07-13 23:45:31.765
1,2024-07-13 23:45:31.760,2024-07-14 05:45:31.760,Pickonimbus 2000,199000,0,2024-07-13 23:45:31.760
2,2024-07-13 23:45:31.645,2024-07-15 23:45:31.645,Terror Helmet,170000,0,2024-07-13 23:45:31.645
3,2024-07-13 23:45:31.568,2024-07-14 05:45:31.568,Cicada Symphony Vinyl,650000,0,2024-07-13 23:45:31.568
4,2024-07-13 23:45:31.531,2024-07-14 05:45:31.531,Gemstone Chamber,6700000,0,2024-07-13 23:45:31.531
...,...,...,...,...,...,...
995,2024-07-13 00:21:25.067,2024-07-14 00:21:25.067,Auspicious Gemstone Gauntlet,500,17000000,2024-07-13 23:40:47.306
996,2024-07-13 23:40:46.602,2024-07-14 05:40:46.602,Blue Gift Backpack Skin,2500000,0,2024-07-13 23:40:46.602
997,2024-07-13 23:40:46.328,2024-07-14 05:40:46.328,Sweet Axe,100000,0,2024-07-13 23:40:46.328
998,2024-07-13 23:40:46.200,2024-07-17 23:40:46.200,Zombie Chestplate,889999,0,2024-07-13 23:40:46.200


In [102]:
def find_item(item_name:str, item_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {item_price}')
    if filtered_df.empty:
        return "Item not on market"
    return filtered_df
find_item('Zombie Chestplate', 1000000)

,start,end,item_name,starting_bid,highest_bid_amount,last_updated
998,2024-07-13 23:40:46.200,2024-07-17 23:40:46.200,Zombie Chestplate,889999,0,2024-07-13 23:40:46.200
